In [1]:
import torch
from torch import nn
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import transforms

try:
    from torchmetrics.functional import pairwise_euclidean_distance
except:
    !pip install torchmetrics

import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
path = "D:\OneDrive - NITT\CODE\Python Scripts\Code_2023\Pics"
import torchvision.models as models
alexnet = models.alexnet(weights=True)
alexnet.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [2]:
preprocess = transforms.Compose([
    transforms.Resize(256),
#     transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
def euc_dist(tensor1,tensor2):
    return (tensor1 - tensor2).pow(2).sum().sqrt().tolist()

In [4]:
def get_activations(model,image_path):
    input_image = Image.open(image_path)

    input_tensor = preprocess(input_image)
    batch_img = torch.unsqueeze(input_tensor, 0)

    output = model(batch_img)
    conv_relu_activations = []
    for indx,layer in enumerate(model.features):
        if type(layer) == nn.ReLU:
            print(f'Conv Activation for: {layer} at {indx} Loaded.')
            activ = torch.flatten(layer(batch_img)[0])
            conv_relu_activations.append(activ[:10].tolist())
            
    fc_relu_activations = []
    for indx,layer in enumerate(model.classifier):
        if type(layer) == nn.ReLU:
            print(f'Fully Connected Activation for: {layer} at {indx} Loaded.')
            activ = torch.flatten(layer(batch_img)[0])
            fc_relu_activations.append(activ[:10].tolist())
            
    return conv_relu_activations,fc_relu_activations

In [5]:
df = pd.DataFrame([])
for image in os.listdir(path):
    image_path = os.path.join(path,image)
    conv,fc = get_activations(alexnet,image_path)
    df[image] = conv + fc
    print(f'{image} processed.')
    print("-"*25)

Conv Activation for: ReLU(inplace=True) at 1 Loaded.
Conv Activation for: ReLU(inplace=True) at 4 Loaded.
Conv Activation for: ReLU(inplace=True) at 7 Loaded.
Conv Activation for: ReLU(inplace=True) at 9 Loaded.
Conv Activation for: ReLU(inplace=True) at 11 Loaded.
Fully Connected Activation for: ReLU(inplace=True) at 2 Loaded.
Fully Connected Activation for: ReLU(inplace=True) at 5 Loaded.
001.jpg processed.
-------------------------
Conv Activation for: ReLU(inplace=True) at 1 Loaded.
Conv Activation for: ReLU(inplace=True) at 4 Loaded.
Conv Activation for: ReLU(inplace=True) at 7 Loaded.
Conv Activation for: ReLU(inplace=True) at 9 Loaded.
Conv Activation for: ReLU(inplace=True) at 11 Loaded.
Fully Connected Activation for: ReLU(inplace=True) at 2 Loaded.
Fully Connected Activation for: ReLU(inplace=True) at 5 Loaded.
002.jpg processed.
-------------------------
Conv Activation for: ReLU(inplace=True) at 1 Loaded.
Conv Activation for: ReLU(inplace=True) at 4 Loaded.
Conv Activation 

In [ ]:
for i in range(len(df)):
    matrix = pairwise_euclidean_distance(torch.tensor(df.iloc[i]))
#     sns.heatmap(matrix)
    plt.matshow(matrix)
    plt.show()

In [6]:
matrix = pairwise_euclidean_distance(torch.tensor(df.iloc[0]))

In [ ]:
plt.matshow(matrix)

In [11]:
d = pd.DataFrame(matrix,columns=os.listdir(path))

In [8]:
df.iloc[0]

001.jpg    [1.1186745166778564, 1.1700488328933716, 0.724...
002.jpg    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
058.jpg    [0.3994349241256714, 0.0, 0.0, 0.0, 0.0, 0.0, ...
064.jpg    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: 0, dtype: object

In [12]:
d.dtypes

001.jpg    float32
002.jpg    float32
058.jpg    float32
064.jpg    float32
dtype: object